In [1]:
import torch
from d2l import torch as d2l

In [ ]:
class Classifier(d2l.Module):  #@save
    """The base class of classification models."""

    def validation_step(self, batch):
        #batch[:-1]表示取batch中除了最后一个元素之外的所有元素，这通常指的是输入特征X（如果batch只包含了特征和标签）。
        # *是Python中的解包操作符，它将batch[:-1]解包成多个独立的参数，然后传递给模型。
        # 这种方式使得方法能够处理不仅仅是特征和标签，还可以处理其他可能传入模型的额外数据。
        Y_hat = self(*batch[:-1])
        # batch[-1]是当前批次的真实标签
        self.plot('loss', self.loss(Y_hat, batch[-1]), train=False)
        self.plot('acc', self.accuracy(Y_hat, batch[-1]), train=False)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=self.lr)

    #提供了一种通用且灵活的方式来计算模型在给定数据上的准确率，无论是获取整体平均准确率还是每个样本的准确性。
    def accuracy(self, Y_hat, Y, averaged=True):
        """Compute the number of correct predictions."""
        # Y_hat: 模型的预测输出，通常是一个具有概率分布形式的张量，其中每行代表一个样本，每列代表一个类别的预测概率。
        # averaged是一个布尔值，指示是否返回所有样本的准确率平均值。默认为True。

        # 这行代码重新塑形Y_hat，确保它是一个二维张量，其中第一维是样本数量，第二维是类别数量。这对于处理在前向传播过程中可能产生的多余维度很有用。
        Y_hat = Y_hat.reshape((-1, Y_hat.shape[-1]))
        # 这行代码找出Y_hat中每行最大值的索引，即模型预测的类别。
        # .argmax(axis=1)沿着每一行进行操作，返回每个样本最高概率类别的索引。.type(Y.dtype)确保预测的数据类型与真实标签Y的类型相匹配。
        preds = Y_hat.argmax(axis=1).type(Y.dtype)
        compare = (preds == Y.reshape(-1)).type(torch.float32)
        return compare.mean() if averaged else compare